In [1]:
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import concurrent.futures
import requests
import threading
import sys

# API Keys (Bybit and Bitget only for this trial)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Telegram and Discord Bot Settings
TELEGRAM_BOT_TOKEN = '7570092528:AAEe-BPQH5LqRq76vxDlxdtsFvhYlU6y7yg'
TELEGRAM_CHAT_ID = '8010042439'
DISCORD_WEBHOOK_URL = 'https://discord.com/api/webhooks/1354874948845568170/_T8lTTaa_w26N0tZr1QJYFb8YkIXB7ct89U_pUuI2ycVEaqtu5WTf9FakHacAkjGJQ-O'

# Alert Functions
def send_telegram_alert(message):
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        data = {"chat_id": TELEGRAM_CHAT_ID, "text": message}
        requests.post(url, data=data)
    except Exception as e:
        print(f"❌ Telegram Error: {e}")

def send_discord_alert(message):
    try:
        requests.post(DISCORD_WEBHOOK_URL, json={"content": message})
    except Exception as e:
        print(f"❌ Discord Error: {e}")

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to Bitget API
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Settings
SPREAD_THRESHOLD = 0.1 / 100
MIN_VOLUME = 50000
CHECK_INTERVAL = 5
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit, 'Bitget': bitget}
futures_markets_cache = {}

# User input with timeout using thread (Windows compatible)
def input_with_timeout(prompt, timeout=10):
    result = {}

    def get_input():
        try:
            result["input"] = input(prompt)
        except Exception:
            result["input"] = None

    thread = threading.Thread(target=get_input)
    thread.daemon = True
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        print("⏱️ No response. Skipping...\n")
        return None
    return result.get("input", None)

# Order execution simulation
def execute_trade(exchange, symbol, side, size, is_limit=True):
    try:
        order_type = 'limit' if is_limit else 'market'
        book = exchange.fetch_order_book(symbol)
        price = book['bids'][0][0] if side == 'sell' else book['asks'][0][0]
        print(f"🚀 Placing {order_type.upper()} {side.upper()} order on {symbol} at {price}")
        return exchange.create_order(symbol, order_type, side, size, price if is_limit else None)
    except Exception as e:
        print(f"❌ Error placing order: {e}")
        return None

# Token filter
def get_tradable_tokens(exchange, exchange_name):
    try:
        print(f"🔍 Fetching tradable tokens for {exchange_name}...")
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [s for s in markets if ":USDT" in s or "-USDT-SWAP" in s]
        for symbol, market in markets.items():
            if not market.get('spot', False):
                continue
            if '/USDT' in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    if market_data.get('quoteVolume', 0) >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error getting tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last")
    except Exception as e:
        print(f"⚠️ Error fetching price for {symbol}: {e}")
        return None

def fetch_prices_concurrently(exchange, symbols):
    prices = {}
    def fetch(symbol):
        prices[symbol] = get_price(exchange, symbol)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(fetch, symbols)
    return prices

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def handle_trade_prompt(exchange, exchange_name, symbol, futures_symbol, spot_price, futures_price, spread, net_profit):
    try:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        alert = f"🔍 [{timestamp}] Arbitrage on {exchange_name}: {symbol}\nSpot: {spot_price:.5f} | Futures: {futures_price:.5f} | Spread: {spread:.4%} | Net: {net_profit:.4%}"
        send_telegram_alert(alert)
        send_discord_alert(alert)

        print(alert)
        response = input_with_timeout("Do you want to execute this trade? (yes/no): ", timeout=10)
        if response and response.lower() == 'yes':
            amount_str = input_with_timeout("How much (in USDT) do you want to trade?: ", timeout=10)
            try:
                size = float(amount_str)
                print(f"⚙️ Executing arbitrage for {symbol} on {exchange_name}")
                first_leg = execute_trade(exchange, symbol, 'buy', size, is_limit=True)
                if first_leg:
                    print("✅ First leg executed. Placing second leg (futures)...")
                    second_leg = execute_trade(exchange, futures_symbol, 'sell', size, is_limit=False)
                    if second_leg:
                        print("🎯 Delta-neutral arbitrage trade completed!")
                    else:
                        print("⚠️ Failed to place second leg.")
            except Exception as e:
                print(f"❌ Error parsing amount or executing trade: {e}")
    except Exception as e:
        print(f"❌ Error in trade prompt thread: {e}")


def check_arbitrage(exchange, exchange_name, symbol, spot_prices, futures_prices):
    try:
        spot_price = spot_prices.get(symbol)
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)
        if not (spot_price and futures_symbol): return
        futures_price = futures_prices.get(futures_symbol)
        if not futures_price: return

        trading_fee = TRADING_FEES.get(exchange_name, 0.001)
        spread = (futures_price - spot_price) / spot_price
        net_profit = spread - (trading_fee * 2)

        if net_profit >= SPREAD_THRESHOLD:
            threading.Thread(target=handle_trade_prompt, args=(exchange, exchange_name, symbol, futures_symbol, spot_price, futures_price, spread, net_profit)).start()
    except Exception as e:
        print(f"❌ Error checking arbitrage: {e}")


def process_tokens(exchange, exchange_name, tokens):
    try:
        futures_list = futures_markets_cache.get(exchange_name, [])
        symbol_map = {t: find_futures_symbol(t, futures_list) for t in tokens}
        symbol_map = {k: v for k, v in symbol_map.items() if v}
        if not symbol_map:
            print(f"⚠️ No spot-futures pairs found on {exchange_name}")
            return
        spot_prices = fetch_prices_concurrently(exchange, list(symbol_map.keys()))
        futures_prices = fetch_prices_concurrently(exchange, list(symbol_map.values()))

        with concurrent.futures.ThreadPoolExecutor() as executor:
            tasks = [executor.submit(check_arbitrage, exchange, exchange_name, t, spot_prices, futures_prices) for t in symbol_map.keys()]
            concurrent.futures.wait(tasks)
    except Exception as e:
        print(f"❌ Error processing tokens: {e}")

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                print(f"\n🔁 Starting scan for {exchange_name}...")
                tokens = get_tradable_tokens(exchange, exchange_name)
                if not tokens:
                    print(f"❌ No tokens on {exchange_name}. Retrying...")
                    time.sleep(10)
                    continue
                process_tokens(exchange, exchange_name, tokens)
            print("✅ Cycle complete. Waiting...\n")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Main loop error: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()



🔁 Starting scan for Bybit...
🔍 Fetching tradable tokens for Bybit...
✅ 414 tradable tokens on Bybit

🔁 Starting scan for Bitget...
🔍 Fetching tradable tokens for Bitget...
🔍 [2025-03-28 23:31:07 WIB] Arbitrage on Bybit: OM/USDT
Spot: 6.19620 | Futures: 6.23340 | Spread: 0.6004% | Net: 0.4004%
⏱️ No response. Skipping...



Do you want to execute this trade? (yes/no):  yes


✅ 573 tradable tokens on Bitget
⚠️ Error fetching price for STX/USDT: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743179565656,"data":null}
⚠️ Error fetching price for METIS/USDT: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743179565696,"data":null}
⚠️ Error fetching price for LOOKS/USDT: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743179565701,"data":null}
⚠️ Error fetching price for KSM/USDT: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743179565752,"data":null}
⚠️ Error fetching price for MAGIC/USDT: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743179565754,"data":null}
⚠️ Error fetching price for LUNA/USDT: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743179565754,"data":null}
⚠️ Error fetching price for APE/USDT: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743179565755,"data":null}
⚠️ Error fetching price for C98/USDT: bitget {"code":"429","msg":"Too Many Requ